In [ ]:
########## Dependencies
##### define UNET weights
model_path = "/mnt/local/data2/Bootsma/2D_CTC/src/analysis/publication_code/weights/00_SEE_TC_UNET.hdf5"

##### define samples to process
img_dir = "/mnt/local/data2/Bootsma/2D_CTC/src/analysis/publication_code/test_data/"

##### define GPU indices to use
GPU_2_use = 1 ##### define GPU indices to use
GPU_ids = [str(GPU_2_use)]  # Specify the GPUs you want to use

##### Dependencies
import os
import sys
import nd2
import tifffile
import numpy as np
import tensorflow as tf
from scipy import ndimage
from keras.models import load_model

sys.path.append('../src/') 
import SEE_TC as ctc

##### load model
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_2_use)
with tf.device(f'/gpu:{str(GPU_2_use)}'):  # Use the specified GPU
    model = load_model(model_path)

In [ ]:
# parse inputs (expects .nd2 images)
files = ctc.parse_nd2_paths(img_dir, "nd2", recursive=True)
files = [s.replace(".nd2", "") for s in files]
print(len(files), files[0])

In [ ]:
for i in range(len(files)):
    print("-------------"+str(i)+"------------")
    file_i = files[i]
    img_path_i = file_i+".nd2"

    ########### check the magnificaiton
    with nd2.ND2File(img_path_i) as ndfile:
        text_unparsed = ndfile.text_info
        cap_settings = text_unparsed['capturing']
        optics = text_unparsed['optics']
    if '10x' in optics:
        input_magnification = 10
        
        if os.path.isfile(img_path_i) == False:
            print("skipping sample ("+file_i+") no input")
            print("###############\n")            
            continue
        
    elif '20x' in optics:
            input_magnification = 20
    elif '40x' in optics:
            input_magnification = 40
    else:
        print("skipping sample ("+file_i+"), unknown magnification")
        print("###############\n")            
        continue
    
    ############# check if already processed
    if os.path.isfile(file_i+".bin_open.tiff"):
            print("skipping sample ("+file_i+"), already processed")
            print("###############\n")            
            continue
    
    ############# READ RAW ARRAY #############
    img_i, c_names = ctc.extract_all_array_nd2(img_path_i)
    img_i = np.transpose(img_i, (1,2,0)) # channel last
    c = next((i for i, s in enumerate(c_names) if "350" in s or "390" in s), -1)
    
    print(file_i)
    print(img_i.shape, c_names, c)
    
    ##### Segment #####   
    # 10x
    if input_magnification == 10:
        bin_i = ctc.segment_image(img_i, c, model, window_size=(32, 32), step_size=16, mode = "GPU", GPU_ids = GPU_ids)            
    # non-10x
    else:
        scale_factor = input_magnification/10 # unet is trained at 10x so scale to fit that
        print("re-scaling input by factor of "+str(scale_factor))
        img_i = ctc.downscale_by_factor_of(img_i, scale_factor)
        print(img_i.shape)
        bin_i = ctc.segment_image(img_i, c, model, window_size=(32, 32), step_size=16, mode = "GPU", GPU_ids = GPU_ids)
        print('scaling back to input size...')
        bin_i = ctc.downscale_by_factor_of(bin_i, 1/scale_factor) # scale binary up to input size            
    
    print("writing binary segmentation array...")
    structuring_element = np.ones((3, 3))
    opened_image = ndimage.binary_opening(bin_i, structure=structuring_element, iterations=2)
    tifffile.imwrite(file_i+".bin_open.tiff", data=opened_image.astype(np.uint16))